<a href="https://colab.research.google.com/github/hukgithub/AI_Makeup/blob/main/AI_Recommended_Makeup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio transformers torch openai-whisper mediapipe opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 19.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.2 MB/s 

In [23]:
import gradio as gr
from transformers import pipeline
import whisper
import cv2
import numpy as np
import mediapipe as mp
import torch
import threading
import time

# Load Whisper model for voice transcription
device = "cuda" if torch.cuda.is_available() else "cpu"
transcribe_model = whisper.load_model("base", device=device)

# Load a pre-trained language model for skincare advice
skincare_model = pipeline("text-generation", model="distilgpt2", device=0 if device == "cuda" else -1)

# Initialize MediaPipe for facial landmark detection
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False)

# Global variable for latest frame
latest_frame = None
capture = True

# Start a new thread for capturing video
def start_video_capture():
    global latest_frame, capture
    cap = cv2.VideoCapture(0)
    while capture:
        ret, img = cap.read()
        if ret:
            latest_frame = img.copy()  # Save the raw frame
        time.sleep(0.1)  # Capture a frame every 100ms
    cap.release()

# Apply virtual makeup to a frame
def apply_virtual_makeup(image):
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for landmark in face_landmarks.landmark:
                x = int(landmark.x * image.shape[1])
                y = int(landmark.y * image.shape[0])
                cv2.circle(image, (x, y), 1, (255, 0, 0), -1)  # Simple "blush" effect
    return image

# Capture a frame with virtual makeup
def capture_and_apply_makeup():
    global latest_frame
    if latest_frame is not None:
        return apply_virtual_makeup(latest_frame)
    return None

# Transcribe voice to text using Whisper
def transcribe_voice(audio):
    transcription = transcribe_model.transcribe(audio)["text"]
    return transcription

# Generate skincare recommendation
def generate_skincare_recommendation(text_input):
    prompt = f"Generate skincare advice for the following concerns: {text_input}"
    response = skincare_model(prompt, max_length=100, num_return_sequences=1)
    return response[0]['generated_text']

# Get skincare advice based on text or voice input
def skincare_consultant(text_input=None, audio_input=None, apply_ai_makeup=False):
    if apply_ai_makeup:
        question_text = "Suggest a makeup look for a glowing, natural appearance."
    else:
        question_text = text_input or "Provide general skincare advice."
    if audio_input:
        question_text = transcribe_voice(audio_input)
    skincare_advice = generate_skincare_recommendation(question_text)
    return skincare_advice

# Start video capture in a separate thread
video_thread = threading.Thread(target=start_video_capture)
video_thread.start()

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# Real-Time AI Makeup and Skincare Consultant")

    with gr.Row():
        video_feed = gr.Video( streaming=True, label="Live Video Feed")  # Use 'camera' for live feed
        capture_image = gr.Image(label="Captured Image with Makeup")

    with gr.Row():
        capture_button = gr.Button("Capture and Apply Makeup")
        capture_button.click(fn=capture_and_apply_makeup, inputs=[], outputs=capture_image)

    with gr.Row():
        text_input = gr.Textbox(label="Type Skincare Question (Optional)")
        audio_input = gr.Audio(type="filepath", label="Ask a Skincare Question (Optional)")

    skincare_advice_output = gr.Textbox(label="Skincare Advice")

    submit_button = gr.Button("Get Advice")
    submit_button.click(
        skincare_consultant,
        inputs=[text_input, audio_input, gr.Checkbox(value=False, visible=False)],  # Hidden checkbox to match inputs
        outputs=[skincare_advice_output]
    )

    # Add button for AI-recommended makeup
    ai_makeup_button = gr.Button("Apply AI Recommended Makeup")
    ai_makeup_button.click(
        skincare_consultant,
        inputs=[gr.Textbox(value="", visible=False), gr.Audio(type="filepath", visible=False), gr.Checkbox(value=True)],  # Set apply_ai_makeup to True
        outputs=[skincare_advice_output]
    )

demo.launch()

# Stop video capture when done
capture = False
video_thread.join()



Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d10c187a9f9b06c7fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
